# Seldon Deployment

Here we will prove that our permission model for seldon works as it should 

In [ ]:
! pip install --user --upgrade pip
! pip install --user --upgrade seldon-core protobuf

*!!!Restart kernel if `seldon-core` or `protobuf` has been installed*

Now let's create a sample Seldon deployment. Super simple! Yet we need just something to try

In [1]:
with open('seldon.yaml', 'w') as f:
    f.write("""
apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/mock_classifier_rest:1.3
          name: classifier
    graph:
      children: []
      endpoint:
        type: REST
      name: classifier
      type: MODEL
    name: example
    replicas: 1
""")
    
! kubectl apply -f 'seldon.yaml'
! kubectl get services

seldondeployment.machinelearning.seldon.io/seldon-model created
NAME                              TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)             AGE
seldon-model-example-classifier   ClusterIP   10.100.47.113    <none>        9000/TCP,9500/TCP   0s
trying-seldon                     ClusterIP   10.100.157.254   <none>        80/TCP              4h44m


First things first! For the beginning let's prove that seldon is actually working by accessing model by it's kubernetes service

In [4]:
! curl -vsd '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' http://seldon-model-example.workspace.svc.cluster.local:8000/api/v1.0/predictions -H "Content-Type: application/json"

*   Trying 10.100.196.144...
* TCP_NODELAY set
* Connected to seldon-model-example.workspace.svc.cluster.local (10.100.196.144) port 8000 (#0)







* upload completely sent off: 39 out of 39 bytes












{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{}}
* Connection #0 to host seldon-model-example.workspace.svc.cluster.local left intact


Now let's use 

In [5]:
! curl -vsd '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}' http://serving.female-kira-813.bubble.superhub.io/seldon/workspace/seldon-model/api/v1.0/predictions -H "Content-Type: application/json"

*   Trying 54.203.69.137...
* TCP_NODELAY set
* Connected to serving.female-kira-813.bubble.superhub.io (54.203.69.137) port 80 (#0)
> POST /seldon/workspace/seldon-model/api/v1.0/predictions HTTP/1.1
> Host: serving.female-kira-813.bubble.superhub.io
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 39
> 
* upload completely sent off: 39 out of 39 bytes
< HTTP/1.1 200 OK
< date: Tue, 05 Jan 2021 22:45:04 GMT
< content-type: application/json
< content-length: 73
< access-control-allow-headers: Accept, Accept-Encoding, Authorization, Content-Length, Content-Type, X-CSRF-Token
< access-control-allow-methods: OPTIONS,POST
< access-control-allow-origin: *
< seldon-puid: b7ccaa75-0ed2-482d-9b43-6835ec7ba201
< x-content-type-options: nosniff
< x-envoy-upstream-service-time: 19
< server: envoy
< 
{"data":{"names":["proba"],"ndarray":[[0.43782349911420193]]},"meta":{}}
* Connection #0 to host serving.female-kira-813.bubble.superhub.io left intact


In [6]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(
    deployment_name="seldon-model",
    gateway_endpoint="serving.female-kira-813.bubble.superhub.io",
    namespace="workspace", debug=True
)

In [7]:
sc.predict(gateway="istio", transport="rest")

send: b'POST /seldon/workspace/seldon-model/api/v1.0/predictions HTTP/1.1\r\nHost: serving.female-kira-813.bubble.superhub.io\r\nUser-Agent: python-requests/2.22.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-Length: 83\r\nContent-Type: application/json\r\n\r\n'
send: b'{"meta": {}, "data": {"tensor": {"shape": [1, 1], "values": [0.0740528698034908]}}}'
reply: 'HTTP/1.1 200 OK\r\n'
header: date: Tue, 05 Jan 2021 22:45:14 GMT
header: content-type: application/json
header: content-length: 95
header: access-control-allow-headers: Accept, Accept-Encoding, Authorization, Content-Length, Content-Type, X-CSRF-Token
header: access-control-allow-methods: OPTIONS,POST
header: access-control-allow-origin: *
header: seldon-puid: 1658104d-0448-476c-8364-b46da8fe6a34
header: x-content-type-options: nosniff
header: x-envoy-upstream-service-time: 8
header: server: envoy


Success:True message:
Request:
meta {
}
data {
  tensor {
    shape: 1
    shape: 1
    values: 0.0740528698034908
  }
}

Response:
{'data': {'names': ['proba'], 'tensor': {'shape': [1, 1], 'values': [0.05506437766858161]}}, 'meta': {}}

In [ ]:
! kubectl delete -f 'seldon.yaml'